In [1]:
# Imports Train
from tweet_data_label import train_data

311
Tweet 1
'This is my f**king country': Racist white woman arrested for attacking passengers on New York bus https://t.co/Rfm0jauh72
_____

Tweet 2
RT @mussie671: Freestyle #3, Fiasco type shiiii https://t.co/0GTJscWORW
_____

Tweet 3
What you think Cole? @JColeNC @JColeDirect https://t.co/oE4Ipvq6hL
_____

Tweet 4
⚡️ “Redskins dominate the Raiders on Sunday Night Football ”https://t.co/4VuHWOs1gX
DATE: Sunday Night 
_____

Tweet 5
RT @NWSSanJuan: 215PM FLASH FLOOD EMERGENCY for A Dam Failure in Isabela Municipality y Quebradillas Municipality in Puerto Rico... #prwx h…
NEIGHBORHOOD: Isabela Municipality NEIGHBORHOOD: Quebradillas Municipality 
TIME: 215PM 
_____

Tweet 6
Posted @chinatown on the corner of 7th &amp; D st. NW, PERFECT WEATHER TO SLIDE UP!!
ADDRESS: 7th &amp; D st. NW 

POSITIVE: Posted @chinatown on the corner of 
_____

Tweet 7
Good Morning DMV, we are @ChinaTown on the corner of 7th &amp; G st NW. Hand crafted burgers, endless toppings, Hand cut fries,  PULL UP !!!


We have missed ALL our fans in #Tysons speared the news we are there for lunch tomorrow #vegan #Egyptian #yummy… https://t.co/uviH5VhAL1

NEGATIVE
_____



In [2]:
# Imports Dependencies
import spacy
import re
import numpy as np

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dropout

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.utils import class_weight

import keras
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten

from sklearn.metrics import confusion_matrix, classification_report, accuracy_score

Using TensorFlow backend.
/Users/farrisismati/anaconda3/envs/nlp_course/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/farrisismati/anaconda3/envs/nlp_course/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/farrisismati/anaconda3/envs/nlp_course/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np

In [3]:
# Quick Validation on Tweet Data
def tweet_data_validate(data):
    print(f'Length: {len(data)}')

In [4]:
tweet_data_validate(train_data)

Length: 311


In [5]:
# Load up spACY library
nlp = spacy.load('en')

In [6]:
affirmation_map = {
    'POSITIVE': 0,
    'NEGATIVE': 1,
    'NULL': 2
}
    
# Organize Data Methods
def tweet_clean(text):
    lower = []
    text = text.replace('&amp;', 'and')

    for token in nlp(text):
        lower.append(token.text.lower())
            
    return lower

# Only target affirmation data
def organize_tweet_data(train_data):
    affirmation_map = {
        'POSITIVE': 0,
        'NEGATIVE': 1,
        'NULL': 1
    }
    tweet_sequences = []
    target_affirm = []

    # Organize tweets & train data into arrays
    for tweet_data in train_data:
        tweet_sequences.append(tweet_clean(tweet_data[0]))
        affirmation = tweet_data[1]['affirmation'][0][0] # Only first affirmation
        target_affirm.append(affirmation_map[affirmation])
        
    return [tweet_sequences, target_affirm]

#  Tokenization Methods
def tokenize_tweets(tokenizer, data):
    tokenizer.fit_on_texts(data)
    
    return tokenizer.texts_to_sequences(data)
    
def create_zeros_array(length):
    zeros_arr = []

    i = 0
    while i < length:
        zeros_arr.append(0)
        i += 1

    return zeros_arr

def find_max_tweet_len(data):
    max_len = 0

    # Find longest sequence, pad everything to its length
    for tweet in data:
        seq_len = len(tweet)
        if seq_len > max_len:
            max_len = seq_len

    return max_len

def pad_array(data, max_len):
    zeros_len = max_len - len(data)
    zeros_arr = create_zeros_array(zeros_len)

    return [*data,*zeros_arr]

In [7]:
# Tokenize & Pad Tweet Data
tweet_sequences, target_affirm = organize_tweet_data(train_data)

tokenizer = Tokenizer()
tokenized_data = tokenize_tweets(tokenizer, tweet_sequences)

max_len = find_max_tweet_len(tokenized_data)

X = [pad_array(data, max_len) for data in tokenized_data]

In [8]:
# Check size of tokenized vocab
vocab_size = len(tokenizer.index_word)

print(f'Vocab Size: {vocab_size}')
print(f'Target Affirm Length: {len(target_affirm)}')

Vocab Size: 1354
Target Affirm Length: 311


In [9]:
# Convert tokenized data to Numpy Array
X = np.array(X)
y = to_categorical(target_affirm)

In [10]:
# Validate Data
print(f'X shape: {X.shape}')
print(f'y shape: {y.shape}')

X shape: (311, 34)
y shape: (311, 2)


In [11]:
# Setup Train/Test Split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.33,random_state=42)

In [12]:
# Validate Data
print('X_train')
print(f'X_train len: {len(X_train)}')
# print(X_train)
print('')
print('y_train')
print(f'y_train len: {len(y_train)}')
# print(y_train)

X_train
X_train len: 208

y_train
y_train len: 208


In [13]:
# Scale Train Data
def scale_data(X_train, x_test):
    scaler_object = MinMaxScaler()
    scaler_object.fit(X_train)
    
    scaled_X_train = scaler_object.transform(X_train)
    scaled_X_test = scaler_object.transform(X_test)
    return [scaled_X_train, scaled_X_test]

scaled_X_train, scaled_X_test = scale_data(X_train, X_test)

In [24]:
# Setup Training Model
def create_model():
    model = Sequential()
    model.add(Dense(12,input_dim=max_len,activation='relu'))
#     model.add(Dropout(0.2))
    model.add(Dense(8, activation='relu'))
    
#     model.add(Dropout(0.2))
    model.add(Dense(2,activation='softmax'))
    model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
    
    return model

In [25]:
model = create_model()

# Review Model
print(model.summary())

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_8 (Dense)              (None, 12)                420       
_________________________________________________________________
dense_9 (Dense)              (None, 8)                 104       
_________________________________________________________________
dense_10 (Dense)             (None, 2)                 18        
Total params: 542
Trainable params: 542
Non-trainable params: 0
_________________________________________________________________
None


In [26]:
# Train - 12/28/20 %45.63
model.fit(scaled_X_train,y_train, epochs=100, batch_size=10,verbose=2, shuffle=True)

Epoch 1/100
 - 0s - loss: 0.7013 - accuracy: 0.3990
Epoch 2/100
 - 0s - loss: 0.6903 - accuracy: 0.5481
Epoch 3/100
 - 0s - loss: 0.6808 - accuracy: 0.5769
Epoch 4/100
 - 0s - loss: 0.6710 - accuracy: 0.6058
Epoch 5/100
 - 0s - loss: 0.6604 - accuracy: 0.6442
Epoch 6/100
 - 0s - loss: 0.6490 - accuracy: 0.6635
Epoch 7/100
 - 0s - loss: 0.6369 - accuracy: 0.6731
Epoch 8/100
 - 0s - loss: 0.6250 - accuracy: 0.6923
Epoch 9/100
 - 0s - loss: 0.6131 - accuracy: 0.6875
Epoch 10/100
 - 0s - loss: 0.6015 - accuracy: 0.7019
Epoch 11/100
 - 0s - loss: 0.5880 - accuracy: 0.7212
Epoch 12/100
 - 0s - loss: 0.5766 - accuracy: 0.7308
Epoch 13/100
 - 0s - loss: 0.5631 - accuracy: 0.7260
Epoch 14/100
 - 0s - loss: 0.5515 - accuracy: 0.7260
Epoch 15/100
 - 0s - loss: 0.5378 - accuracy: 0.7356
Epoch 16/100
 - 0s - loss: 0.5255 - accuracy: 0.7452
Epoch 17/100
 - 0s - loss: 0.5121 - accuracy: 0.7548
Epoch 18/100
 - 0s - loss: 0.5012 - accuracy: 0.7740
Epoch 19/100
 - 0s - loss: 0.4899 - accuracy: 0.7740
Ep

In [27]:
# Predictions
model.predict_classes(scaled_X_test)
predictions = model.predict_classes(scaled_X_test)

print('Results')
print(f'X : {predictions}')
print(f'Y : {y_test.argmax(axis=1)}\n')

print('Confusion Matrix')
print(print(f'Positive, Negative'))
print(f'{confusion_matrix(y_test.argmax(axis=1),predictions)}\n')

print('Classification Report')
print(classification_report(y_test.argmax(axis=1),predictions))

# 12/28/20 %42
print('Accuracy Score')
print(accuracy_score(y_test.argmax(axis=1),predictions))

Results
X : [0 0 1 1 0 0 0 1 0 1 0 0 0 1 0 1 0 1 0 1 1 0 0 1 1 1 1 0 1 1 0 1 0 1 1 0 1
 0 0 1 1 1 1 1 0 0 1 0 0 1 1 0 0 0 1 0 1 1 1 1 0 1 1 1 1 0 0 1 1 0 1 0 0 0
 0 0 0 1 0 0 0 0 1 0 0 0 0 1 0 1 0 1 0 1 1 1 0 0 0 0 1 1 0]
Y : [0 0 1 0 0 0 0 0 1 1 1 1 0 1 0 0 0 1 0 1 1 1 0 1 1 1 0 0 0 1 1 1 0 1 0 1 1
 0 0 1 1 0 0 1 0 0 1 0 0 1 1 1 0 0 0 1 1 0 0 0 0 0 0 1 1 0 1 1 0 0 1 0 0 0
 0 0 0 0 0 1 0 1 1 1 0 0 1 1 0 1 0 1 0 1 0 1 0 0 0 1 1 1 1]

Confusion Matrix
Positive, Negative
None
[[39 17]
 [15 32]]

Classification Report
              precision    recall  f1-score   support

           0       0.72      0.70      0.71        56
           1       0.65      0.68      0.67        47

    accuracy                           0.69       103
   macro avg       0.69      0.69      0.69       103
weighted avg       0.69      0.69      0.69       103

Accuracy Score
0.6893203883495146


In [19]:
tokenized_tweets = tokenize_tweets(tokenizer, ["No food today"])
predict_tweets = [pad_array(data, max_len) for data in tokenized_tweets]

scaler_object = MinMaxScaler()
scaler_object.fit(X_test)
scale_test = scaler_object.transform(predict_tweets)
    
prediction = model.predict_classes(scale_test)

print(prediction)

if prediction[0] == 0:
    print('Positive Affirmation')
else:
    print('Negative or no Affirmation')

# Figure out how to predict
# Add More data
# See accuracy

# Create NER model
# Combine this model with NER Model in an API to return data
# Use in Lambda func

[0]
Positive Affirmation
